# TEST

In [2]:
%run __init__.py

In [4]:
import pandas as pd

GIT_FILE_PATH = os.path.join(NOTEBOOK_1_RESULTS_DIR, 'git_dataframe.pkl')

git_df = pd.read_pickle(GIT_FILE_PATH)

/home/alejandro/.envs/edma/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [5]:
git_repositories = git_df['full_text_cleaned'].values

In [6]:
MAX_CHAR_LENGTH = 100000

git_repositories = [r[:MAX_CHAR_LENGTH] for r in git_repositories]

In [26]:
git_repositories[46]

'Foray into ASP.NET Core (Fedora 27). expert-pancake-aspnetcoreapp# expert-pancake-aspnetcoreapp. jshint latedef: false placement == \'right\' jslint white: true, browser: true, onevar: true, undef: true, nomen: true, eqeqeq: true, plusplus: true, bitwise: true, regexp: true, newcap: true, immed: true, strict: false global document: false, jQuery: false * Dutch bank account numbers (not \'giro\' numbers) have 9 digits * and pass the \'11 check\'. * We accept the notation with spaces, as that is common. * acceptable: 123456789 or 12 34 56 789 * * BIC is the business identifier code (ISO 9362). This BIC check is not a guarantee for authenticity. * * BIC pattern: BBBBCCLLbbb (8 or 11 characters long; bbb is optional) * * BIC definition in detail: * - First 4 characters - bank code (only letters) * - Next 2 characters - ISO 3166-1 alpha-2 country code (only letters) * - Next 2 characters - location code (letters and digits) * a. shall not start with \'0\' or \'1\' * b. second character mus

In [8]:
from collections import Counter

from sklearn.base import TransformerMixin, BaseEstimator
from spacy import displacy


class NamedEntityRecognizer(BaseEstimator, TransformerMixin):
    def __init__(self, spacy_model, disable=None, min_entity_counts=None,
                 max_entities=None):
        self.nlp = spacy_model.load()
        self.disable = disable if disable is not None else []
        self.min_entity_counts = min_entity_counts
        self.max_entities = max_entities
    
    def fit(self, X, y=None):
        return self

    def transform(self, X, *args, **kwargs):
        entities_texts = [self.get_entities(text) for text in X]
        if self.min_entity_counts is None:
            return entities_texts
        
        return [[entity_label 
                for entity_label, entity_count in Counter(entities_text).most_common(self.max_entities)
                if entity_count >= self.min_entity_counts]
                for entities_text in entities_texts]
    
    def get_entities(self, text):
        doc = self.nlp(text)
        return [x.text for x in doc.ents 
                if x.label_ not in self.disable
                and len(x.text) > 2]
    
    def get_most_common_entities(self, text, n=10):
        entities = self.get_entities(text)
        return Counter(entities).most_common(n)
    
    def visualize_entities(self, text, jupyter=True):
        doc = self.nlp(text)
        displacy.render(doc, jupyter=jupyter, style='ent')

In [61]:
import en_core_web_md

ner = NamedEntityRecognizer(en_core_web_md)

ModuleNotFoundError: No module named 'en_score_sci_lg'

In [62]:
import en_core_sci_lg

ner_2 = NamedEntityRecognizer(en_core_sci_lg)

In [103]:
text = git_repositories[10]

ents = ner.transform([text])
ents

[['Anaconda', 'Anaconda', 'Cloud', 'conda', 'PDF']]

In [104]:
ner.visualize_entities(text)

In [105]:
ner_2.transform([text])

[['package',
  'R package',
  'evaluate',
  'reliability',
  'bioinformatic metrics',
  'package',
  'evaluate',
  'reliability',
  'bioinformatic',
  'stability',
  'goodness',
  'classifications',
  'metrics',
  'method',
  'measurements',
  'metrics',
  'dataset',
  'evaluates',
  'reliability',
  'metrics',
  'analyses',
  'Correlations',
  'Stability',
  'Goodness',
  'classifications',
  'version',
  'release',
  'bioconductor repository',
  'development',
  'bioconductor repository',
  'Anaconda evaluomeR',
  'Anaconda Cloud',
  'package',
  'manual',
  'PDF']]

In [106]:
ner_2.visualize_entities(text)